# Data Exploration

## Initializing

In [63]:
import os
import csv
import pandas as pd
import ipywidgets as widgets
from IPython.display import display, HTML, clear_output
import time
import dask.dataframe as dd
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib.ticker as ticker
import numpy as np
import dash
from dash import dcc, html
import calendar
from dash.dependencies import Input, Output
from datetime import datetime
import folium

In [64]:
# Set the working directory for fahrzeiten
fahrzeiten_dir = '../raw_data/fahrzeiten_2022/'

# Create the directories if they don't exist
os.makedirs(f'../figures/exploration/{fahrzeiten_dir.split("/")[-2]}', exist_ok=True)

In [65]:
fahrzeiten_filenames = [filename for filename in os.listdir(fahrzeiten_dir) if filename.endswith('.csv') and filename.startswith('Fahrzeiten_SOLL_IST')]
fahrzeiten_head = pd.read_csv(fahrzeiten_dir + fahrzeiten_filenames[0], nrows=0).columns

# Define the data types for each column
dtypes = {
    'linie': 'int16',
    'richtung': 'int8',
    'betriebsdatum': 'object',
    'fahrzeug': 'int32',
    'kurs': 'int16',
    'seq_von': 'int32',
    'halt_diva_von': 'int32',
    'halt_punkt_diva_von': 'int32',
    'halt_kurz_von1': 'object',
    'datum_von': 'object',
    'soll_an_von': 'int32',
    'ist_an_von': 'int32',
    'soll_ab_von': 'int32',
    'ist_ab_von': 'int32',
    'seq_nach': 'int32',
    'halt_diva_nach': 'int32',
    'halt_punkt_diva_nach': 'int32',
    'halt_kurz_nach1': 'object',
    'datum_nach': 'object',
    'soll_an_nach': 'int32',
    'ist_an_nach1': 'int32',
    'soll_ab_nach': 'int32',
    'ist_ab_nach': 'int32',
    'fahrt_id': 'int64',
    'fahrweg_id': 'int64',
    'fw_no': 'int16',
    'fw_typ': 'int8',
    'fw_kurz': 'object',
    'fw_lang': 'object',
    'umlauf_von': 'int64',
    'halt_id_von': 'int64',
    'halt_id_nach': 'int64',
    'halt_punkt_id_von': 'int64',
    'halt_punkt_id_nach': 'int64'
}

def get_fahrzeiten_dask_df(filename, columns=None):
    if columns is None:
        columns = fahrzeiten_head
    df = dd.read_csv(fahrzeiten_dir + filename, usecols=columns, dtype=dtypes)
    return df.compute()

fahrzeiten_dfs = {filename: get_fahrzeiten_dask_df(filename) for filename in fahrzeiten_filenames}

# Function to get the data from the csv files and return a dataframe
def csv_to_df(filepath):
    try:
        return pd.read_csv(filepath, sep=',')
    except pd.errors.ParserError:
        return pd.read_csv(filepath, sep=';')

passagierfrequenz_df = csv_to_df('../raw_data/passagierfrequenz.csv')
haltestelle_df = csv_to_df(fahrzeiten_dir + 'haltestelle.csv')
haltepunkt_df = csv_to_df(fahrzeiten_dir + 'haltepunkt.csv')

dataframes = {
    'passagierfrequenz': passagierfrequenz_df,
    f'haltestelle ({fahrzeiten_dir.split("/")[-2].split("_")[1]})': haltestelle_df,
    f'haltepunkt ({fahrzeiten_dir.split("/")[-2].split("_")[1]})': haltepunkt_df
}

## Displaying the data

### Fahrzeiten CSVs

In [66]:
"""
This function displays a UI for exploring CSV files in the '../raw_data/fahrzeiten' directory.
It provides a dropdown to select a file, and 'Head', 'Previous', 'Next', and 'Tail' buttons to navigate through the data.
The data is displayed in a DataFrame format, showing 10 rows at a time.
"""
def show_all_fahrzeiten_csvs():
    start = 0
    df = None

    # Create UI elements
    head_button = widgets.Button(description='Head')
    prev_button = widgets.Button(description='Previous')
    next_button = widgets.Button(description='Next')
    tail_button = widgets.Button(description='Tail')
    output = widgets.Output()
    column_dropdown = widgets.Dropdown(options=df.columns if df is not None else [])
    search_input = widgets.Text(value='', placeholder='Type something', description='Search:', disabled=False)
    search_button = widgets.Button(description='Search')
    dimensions_label = widgets.HTML()
    not_checkbox = widgets.Checkbox(value=False, description='NOT', layout=widgets.Layout(width='auto'))
    describe_button = widgets.Button(description='Describe')
    
    # Function to load CSV file into DataFrame
    def show_csv(button_instance=None):
        nonlocal start, df
        start = 0
        df = fahrzeiten_dfs[filenames[dropdown.value]]
        column_dropdown.options = df.columns
        show_output()

    # Event handlers for button clicks
    def on_head_button_clicked(b):
        nonlocal start
        start = 0
        show_output()

    def on_prev_button_clicked(b):
        nonlocal start
        start = max(0, start-10)
        show_output()

    def on_next_button_clicked(b):
        nonlocal start
        start = min(len(df)-10, start+10)
        show_output()

    def on_tail_button_clicked(b):
        nonlocal start
        start = len(df)-10
        show_output()

    # Function to display DataFrame in output widget
    def show_output():
        with output:
            output.clear_output()
            display(HTML('<div style="overflow-x: auto; white-space: nowrap;">' 
                        + df.iloc[start:start+10].to_html() + '</div>'))
            # Update the dimensions label
            dimensions_label.value = f'<h4>Dimensions: {df.shape}</h4>'
    
    # Function to show search results
    def show_search(button_instance=None):
        nonlocal df
        if search_input.value:
            if not_checkbox.value:
                df = df[~df[column_dropdown.value].astype(str).str.contains(search_input.value)]
            else:
                df = df[df[column_dropdown.value].astype(str).str.contains(search_input.value)]
        show_output()
    
    # Function to show description
    def show_description(button_instance=None):
        with output:
            output.clear_output()
            if df is not None:
                desc_df = df.describe()
                desc_df = desc_df.applymap(lambda x: '{:.0f}'.format(x) if x == int(x) else '{:.4f}'.format(x))
                display(HTML('<div style="overflow-x: auto; white-space: nowrap;">' 
                        + desc_df.to_html() + '</div>'))
            else:
                display(HTML('<p style="color: red;">Please first select "Show".</p>'))

    # Get list of CSV files
    filenames = {f'{filename[26:28]}.{filename[24:26]}.{filename[20:24]} bis {filename[35:37]}.{filename[33:35]}.{filename[29:33]}': filename for filename in fahrzeiten_filenames}
    
    # Create dropdown and show button
    dropdown = widgets.Dropdown(options=list(filenames.keys()))
    show_button = widgets.Button(description='Show')

    # Display UI elements
    title = widgets.HTML('<h2 style="text-align: center;">Fahrzeiten: SOLL und IST</h2>')
    box_layout = widgets.Layout(display='flex', justify_content='center')
    display(
        widgets.VBox(
            [
                title, widgets.HBox([dropdown, show_button, describe_button], layout=box_layout),
                widgets.HBox([not_checkbox, column_dropdown, search_input, search_button], layout=box_layout),
                output,
                widgets.HBox([dimensions_label], layout=widgets.Layout(justify_content='flex-start')),
                widgets.HBox([head_button, prev_button, next_button, tail_button], layout=box_layout)
            ],
            layout=box_layout
        )
    )
    
    # Attach event handlers to buttons
    show_button.on_click(show_csv)
    describe_button.on_click(show_description)
    head_button.on_click(on_head_button_clicked)
    prev_button.on_click(on_prev_button_clicked)
    next_button.on_click(on_next_button_clicked)
    tail_button.on_click(on_tail_button_clicked)
    search_button.on_click(show_search)

In [67]:
# show_all_fahrzeiten_csvs()

### Other CSVs

In [68]:
"""
This function provides a user interface for exploring CSV files in the '../raw_data' directory.
The user can navigate through the data using 'Head', 'Previous', 'Next', and 'Tail' buttons, which display the data in chunks of 10 rows at a time.
"""
def show_other_csvs():
    start = 0
    df = None

    # Create UI elements
    head_button = widgets.Button(description='Head')
    prev_button = widgets.Button(description='Previous')
    next_button = widgets.Button(description='Next')
    tail_button = widgets.Button(description='Tail')
    output = widgets.Output()
    column_dropdown = widgets.Dropdown(options=df.columns if df is not None else [])
    search_input = widgets.Text(value='', placeholder='Type something', description='Search:', disabled=False)
    search_button = widgets.Button(description='Search')
    dimensions_label = widgets.HTML()
    not_checkbox = widgets.Checkbox(value=False, description='NOT', layout=widgets.Layout(width='auto'))
    describe_button = widgets.Button(description='Describe')

    # Function to load DataFrame from the list
    def show_df(button_instance=None):
        nonlocal start, df
        start = 0
        df = dataframes[dropdown.value]
        column_dropdown.options = df.columns
        show_output()

    # Event handlers for button clicks
    def on_head_button_clicked(b):
        nonlocal start
        start = 0
        show_output()

    def on_prev_button_clicked(b):
        nonlocal start
        start = max(0, start-10)
        show_output()

    def on_next_button_clicked(b):
        nonlocal start
        start = min(len(df)-10, start+10)
        show_output()

    def on_tail_button_clicked(b):
        nonlocal start
        start = len(df)-10
        show_output()

    # Function to display DataFrame in output widget
    def show_output():
        with output:
            output.clear_output()
            display(HTML('<div style="overflow-x: auto; white-space: nowrap;">' 
                        + df.iloc[start:start+10].to_html() + '</div>'))
            # Update the dimensions label
            dimensions_label.value = f'<h4>Dimensions: {df.shape}</h4>'

    # Function to show search results
    def show_search(button_instance=None):
        nonlocal df
        if search_input.value:
            if not_checkbox.value:
                df = df[~df[column_dropdown.value].astype(str).str.contains(search_input.value)]
            else:
                df = df[df[column_dropdown.value].astype(str).str.contains(search_input.value)]
        show_output()
    
    # Function to show description
    def show_description(button_instance=None):
        with output:
            output.clear_output()
            if df is not None:
                desc_df = df.describe()
                desc_df = desc_df.applymap(lambda x: '{:.0f}'.format(x) if x == int(x) else '{:.4f}'.format(x))
                display(HTML('<div style="overflow-x: auto; white-space: nowrap;">' 
                        + desc_df.to_html() + '</div>'))
            else:
                display(HTML('<p style="color: red;">Please first select "Show".</p>'))

    # Get list of dataframe names
    df_names = list(dataframes.keys())
    
    # Create dropdown and show button
    dropdown = widgets.Dropdown(options=df_names)
    show_button = widgets.Button(description='Show')

    # Display UI elements
    title = widgets.HTML('<h2 style="text-align: center;">Other CSVs</h2>')
    box_layout = widgets.Layout(display='flex', justify_content='center')
    display(
        widgets.VBox(
            [
                title, widgets.HBox([dropdown, show_button, describe_button], layout=box_layout),
                widgets.HBox([not_checkbox, column_dropdown, search_input, search_button], layout=box_layout),
                output,
                widgets.HBox([dimensions_label], layout=widgets.Layout(justify_content='flex-start')),
                widgets.HBox([head_button, prev_button, next_button, tail_button], layout=box_layout)
            ],
            layout=box_layout
        )
    )
    
    # Attach event handlers to buttons
    show_button.on_click(show_df)
    describe_button.on_click(show_description)
    head_button.on_click(on_head_button_clicked)
    prev_button.on_click(on_prev_button_clicked)
    next_button.on_click(on_next_button_clicked)
    tail_button.on_click(on_tail_button_clicked)
    search_button.on_click(show_search)

In [69]:
# show_other_csvs()

## Exploring the data

### Wie viele ... gibt es? (Einzigartige Werte)
---
---

#### Frage:
Wie viele Bahnen (linie) gibt es in den Fahrzeiten CSVs?

In [70]:
def create_bar_chart_type_of_line(save=False):
    unique_linien = set()
    for df in fahrzeiten_dfs.values():
        unique_linien.update(df['linie'].unique())

    # Count the number of each type of line
    tram_count = len([linie for linie in unique_linien if 2 <= linie <= 17])
    bus_count = len([linie for linie in unique_linien if 29 <= linie <= 916])
    bergbahn_count = len([linie for linie in unique_linien if 18 <= linie <= 28])
    other_count = len(unique_linien) - tram_count - bus_count - bergbahn_count
    
    # Create a DataFrame with counts and corresponding labels
    df_linien_counts = pd.DataFrame({
        'Type of Line': ['Tram', 'Bus', 'Bergbahn', 'Other'],
        'Count': [tram_count, bus_count, bergbahn_count, other_count]
    })

    # Create the bar chart
    fig, ax = plt.subplots(figsize=(6, 6))
    bars = sns.barplot(x=df_linien_counts['Type of Line'], y=df_linien_counts['Count'], ax=ax, palette='Set2', zorder=2)
    
    # Add the counts above the bars
    for bar in bars.patches:
        ax.text(bar.get_x() + bar.get_width() / 2, bar.get_height(), f'{int(bar.get_height())}', ha='center', va='bottom')

    ax.yaxis.set_major_formatter(ticker.FuncFormatter(lambda x, pos: '{:,.0f}'.format(x)))
    plt.title(f'Count of Each Type of Line in Fahrzeiten CSVs ({fahrzeiten_dir.split("/")[-2].split("_")[1]})')
    plt.xlabel('Type of Line')
    plt.ylabel('Count')
    plt.grid(axis='y', zorder=1)

    if save:
        plt.savefig(f'../figures/exploration/{fahrzeiten_dir.split("/")[-2]}/type_of_line_count.png', bbox_inches='tight')
        plt.close()

In [71]:
create_bar_chart_type_of_line(save=True)

c:\Users\tklas\anaconda3\envs\new_base\Lib\site-packages\seaborn\_oldcore.py:1498: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
  if pd.api.types.is_categorical_dtype(vector):
c:\Users\tklas\anaconda3\envs\new_base\Lib\site-packages\seaborn\_oldcore.py:1498: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
  if pd.api.types.is_categorical_dtype(vector):
c:\Users\tklas\anaconda3\envs\new_base\Lib\site-packages\seaborn\_oldcore.py:1498: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
  if pd.api.types.is_categorical_dtype(vector):


#### Antwort:
Es gibt 73 Linien. Siehe type_of_line_count.png

---

#### Frage:
Gibt es mehrere Bahnunternehmen?

In [72]:
bahnhofseigner_counts = passagierfrequenz_df["isb_gi"].value_counts().to_dict()
df_bahnhofseigner_counts = pd.DataFrame(
    bahnhofseigner_counts.items(), columns=["Station Owner", "Number of Stations"]
)

expanded_df = passagierfrequenz_df["evu_ef_itf"].str.split(',', expand=True).stack().str.strip()
evu_counts = expanded_df.value_counts().to_dict()
df_evu_counts = pd.DataFrame(
    evu_counts.items(), columns=["Company", "Number of Stations"]
)

max_value = max(df_bahnhofseigner_counts["Number of Stations"].max(), df_evu_counts["Number of Stations"].max()) + 80

def create_bar_chart_bahnhofseigner_counts(save=False):
    # Create the horizontal bar chart
    fig, ax = plt.subplots(figsize=(10, 6))
    bars = sns.barplot(
        y=df_bahnhofseigner_counts["Station Owner"],
        x=df_bahnhofseigner_counts["Number of Stations"],
        order=df_bahnhofseigner_counts["Station Owner"].value_counts().index,
        ax=ax,
        palette='Set2',
        zorder=2
    )
    ax.set_xlim(0, max_value)

    # Add the number of stations on top of each bar
    for bar in bars.patches:
        ax.text(
            bar.get_width(),
            bar.get_y() + bar.get_height() / 2,
            f'{int(bar.get_width())}',
            ha='left',
            va='center'
        )

    ax.xaxis.set_major_formatter(
        ticker.FuncFormatter(lambda x, pos: "{:,.0f}".format(x))
    )
    plt.title("Number of Stations per Owner in Passagierfrequenz CSV")
    plt.ylabel("")
    plt.xlabel("Number of Stations")
    plt.grid(axis="x", zorder=1)

    if save:
        plt.savefig("../figures/exploration/bahnhofseigner_count.png", bbox_inches="tight")
        plt.close()
        
def create_bar_chart_eisenbahnunternehmen_counts(save=False):
    # Create the horizontal bar chart
    fig, ax = plt.subplots(figsize=(10, 6))
    bars = sns.barplot(
        y=df_evu_counts["Company"],
        x=df_evu_counts["Number of Stations"],
        ax=ax,
        palette='Set2',
        zorder=2
    )
    ax.set_xlim(0, max_value)
    
    # Add the number of stations on top of each bar
    for bar in bars.patches:
        ax.text(
            bar.get_width(),
            bar.get_y() + bar.get_height() / 2,
            f'{int(bar.get_width())}',
            ha='left',
            va='center'
        )

    ax.xaxis.set_major_formatter(
        ticker.FuncFormatter(lambda x, pos: "{:,.0f}".format(x))
    )
    plt.title("Number of Stations per Company in Passagierfrequenz CSV")
    plt.ylabel("")
    plt.xlabel("Number of Stations")
    plt.grid(axis="x", zorder=1)

    if save:
        plt.savefig("../figures/exploration/eisenbahnunternehmen_count.png", bbox_inches="tight")
        plt.close()

In [73]:
# create_bar_chart_bahnhofseigner_counts(save=True)
# create_bar_chart_eisenbahnunternehmen_counts(save=True)

c:\Users\tklas\anaconda3\envs\new_base\Lib\site-packages\seaborn\_oldcore.py:1498: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
  if pd.api.types.is_categorical_dtype(vector):
c:\Users\tklas\anaconda3\envs\new_base\Lib\site-packages\seaborn\_oldcore.py:1498: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
  if pd.api.types.is_categorical_dtype(vector):
c:\Users\tklas\anaconda3\envs\new_base\Lib\site-packages\seaborn\_oldcore.py:1498: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
  if pd.api.types.is_categorical_dtype(vector):
c:\Users\tklas\anaconda3\envs\new_base\Lib\site-packages\seaborn\_oldcore.py:1498: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(d

#### Antwort:
Siehe bahnhofseigner_count.png und bahnhofs_eisenbahnunternehmen_count.png

---

#### Frage:
Wie viele Haltestellen gibt es in Haltestellen.csv?

In [74]:
print(f'Unique Haltestellen: {len(haltestelle_df["halt_id"].unique())}')

Unique Haltestellen: 769


#### Antwort:
Es gibt 769 Haltestellen.

---

#### Frage:
Wie viele Städte gibt es in den Fahrzeiten CSVs?

In [75]:
unique_staedte = set()
for df in fahrzeiten_dfs.values():
    halt_id_von = df['halt_id_von'].unique()
    halt_id_nach = df['halt_id_nach'].unique()
    halt_id = np.unique(np.concatenate((halt_id_von, halt_id_nach)))
    haltestellen_name = haltestelle_df[haltestelle_df['halt_id'].isin(halt_id)]['halt_lang'].unique()
    stadt_name = map(lambda x: x.split(',')[0], haltestellen_name)
    unique_staedte.update(stadt_name)
unique_staedte = sorted(unique_staedte)

print(f'Einzigartige Städte: {len(unique_staedte)}')
print(unique_staedte)

Einzigartige Städte: 50
['Adliswil', 'Aesch bei Maur', 'Benglen', 'Bergdietikon', 'Binz bei Maur', 'Birmensdorf ZH', 'Dietikon', 'Dübendorf', 'Ebmatingen', 'Effretikon', 'Fahrweid', 'Forch', 'Fällanden', 'Geroldswil', 'Glanzenberg', 'Glattbrugg', 'Glattpark', 'Gockhausen', 'Itschnach', 'Kilchberg ZH', 'Killwangen', 'Kindhausen AG', 'Kloten Balsberg', 'Küsnacht ZH', 'Maur', 'Oberengstringen', 'Oetwil a.d.L.', 'Oetwil an der Limmat', 'Pfaffhausen', 'Rümlang', 'Rüschlikon', 'Scheuren', 'Schlieren', 'Schwerzenbach ZH', 'Spreitenbach', 'Unterengstringen', 'Urdorf', 'Urdorf Weihermatt', 'Volketswil', 'Waldburg', 'Wallisellen', 'Weiningen ZH', 'Wädenswil', 'Zch', 'Zollikerb.', 'Zollikerberg', 'Zollikon', 'Zumikon', 'Zürich', 'Zürich Flughafen']


#### Antwort:
Es gibt zwar laut Ausgabe 50 Städte, aber manche Einträge sind anders geschriebene Städte, die eigentlich die gleiche Stadt sind. 

---

#### Frage:
Wie viele Einträge gibt es in der Passagierfrequenz CSV pro Stadt pro Kategorie?

In [78]:
def create_bar_chart_avg_passenger_counts(save=False, values=10):
    # Combine the two dataframes
    combined = passagierfrequenz_df[passagierfrequenz_df["jahr_annee_anno"].isin([2018, 2022])].copy()

    # Create a dictionary of categories
    categories = {
        'dtv_tjm_tgm': 'Whole Week',
        'dwv_tmjo_tfm': 'Work Week',
        'dnwv_tmjno_tmgnl': 'Non-Work Days and Holidays'
    }

    # Create a bar chart for each category
    for category, category_name in categories.items():
        
        # Create a new column that contains the maximum value for each station
        combined['max_value'] = combined.groupby('bahnhof_gare_stazione')[category].transform('max')

        # Identify the top stations
        top_stations = combined.sort_values('max_value', ascending=False)['bahnhof_gare_stazione'].unique()[:values]

        # Filter the original data for the top stations
        combined_top = combined[combined['bahnhof_gare_stazione'].isin(top_stations)]
        combined_top['jahr_annee_anno'] = combined_top['jahr_annee_anno'].astype(str)
        # Specify the order of the stations
        combined_top['bahnhof_gare_stazione'] = pd.Categorical(combined_top['bahnhof_gare_stazione'], categories=top_stations.tolist(), ordered=True)

        fig, ax = plt.subplots(figsize=(10, 6))
        sns.barplot(y=combined_top['bahnhof_gare_stazione'], x=combined_top[category], hue=combined_top['jahr_annee_anno'], ax=ax, palette='Set2')
        ax.xaxis.set_major_formatter(ticker.FuncFormatter(lambda x, pos: '{:,.0f}'.format(x)))
        plt.title(f'Top {values} Most Frequent Stations in Passagierfrequenz CSV ({category_name})')
        plt.ylabel('')
        plt.xlabel('Average Number of Passengers')
        plt.grid(axis='x')

        # Modify the legend name and position
        ax.legend(title='Year', loc='lower right')

        if save:
            plt.savefig(f'../figures/exploration/avg_passenger_count_per_station_{category}.png', bbox_inches='tight')
            plt.close()

In [79]:
# pd.options.mode.chained_assignment = None
# create_bar_chart_avg_passenger_counts(save=True, values=20)

c:\Users\tklas\anaconda3\envs\new_base\Lib\site-packages\seaborn\_oldcore.py:1498: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
  if pd.api.types.is_categorical_dtype(vector):
c:\Users\tklas\anaconda3\envs\new_base\Lib\site-packages\seaborn\_oldcore.py:1498: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
  if pd.api.types.is_categorical_dtype(vector):
c:\Users\tklas\anaconda3\envs\new_base\Lib\site-packages\seaborn\categorical.py:641: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  grouped_vals = vals.groupby(grouper)
c:\Users\tklas\anaconda3\envs\new_base\Lib\site-packages\seaborn\categorical.py:641: FutureWarning: 

c:\Users\tklas\anaconda3\envs\new_base\Lib\site-packages\seaborn\_oldcore.py:1498: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
  if pd.api.types.is_categorical_dtype(vector):
c:\Users\tklas\anaconda3\envs\new_base\Lib\site-packages\seaborn\_oldcore.py:1498: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
  if pd.api.types.is_categorical_dtype(vector):
c:\Users\tklas\anaconda3\envs\new_base\Lib\site-packages\seaborn\categorical.py:641: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  grouped_vals = vals.groupby(grouper)
c:\Users\tklas\anaconda3\envs\new_base\Lib\site-packages\seaborn\categorical.py:641: FutureWarning: 

In [82]:
def create_bar_chart_avg_passenger_counts_v2(save=False, values=10):
    # Fill the missing values in the jahr_annee_anno column with 2018
    temp = passagierfrequenz_df.copy()
    
    temp['jahr_annee_anno'].fillna(2018, inplace=True)
    
    # Combine the two dataframes
    combined = temp[temp["jahr_annee_anno"].isin([2018, 2022])].copy()

    # Create a dictionary of categories
    categories = {
        'dtv_tjm_tgm': 'Whole Week',
        'dwv_tmjo_tfm': 'Work Week',
        'dnwv_tmjno_tmgnl': 'Non-Work Days and Holidays'
    }

    # Create a bar chart for each category
    for category, category_name in categories.items():
        # Create a new column that contains the maximum value for each station
        combined['max_value'] = combined.groupby('bahnhof_gare_stazione')[category].transform('max')

        # Identify the top stations
        top_stations = combined.sort_values('max_value', ascending=False)['bahnhof_gare_stazione'].unique()[:values]

        # Filter the original data for the top stations
        combined_top = combined[combined['bahnhof_gare_stazione'].isin(top_stations)]
        combined_top['jahr_annee_anno'] = combined_top['jahr_annee_anno'].astype(str)
        # Specify the order of the stations
        combined_top['bahnhof_gare_stazione'] = pd.Categorical(combined_top['bahnhof_gare_stazione'], categories=top_stations.tolist(), ordered=True)

        fig, ax = plt.subplots(figsize=(10, 6))
        sns.barplot(y=combined_top['bahnhof_gare_stazione'], x=combined_top[category], hue=combined_top['jahr_annee_anno'], ax=ax, palette='Set2')
        ax.xaxis.set_major_formatter(ticker.FuncFormatter(lambda x, pos: '{:,.0f}'.format(x)))
        plt.title(f'Top {values} Most Frequent Stations in Passagierfrequenz CSV ({category_name})')
        plt.ylabel('')
        plt.xlabel('Average Number of Passengers')
        plt.grid(axis='x')

        # Modify the legend name and position
        ax.legend(title='Year', loc='lower right')

        if save:
            plt.savefig(f'../figures/exploration/avg_passenger_count_per_station_{category}_v2.png', bbox_inches='tight')
            plt.close()

In [83]:
# create_bar_chart_avg_passenger_counts_v2(save=True, values=20)
# pd.options.mode.chained_assignment = 'warn'

c:\Users\tklas\anaconda3\envs\new_base\Lib\site-packages\seaborn\_oldcore.py:1498: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
  if pd.api.types.is_categorical_dtype(vector):
c:\Users\tklas\anaconda3\envs\new_base\Lib\site-packages\seaborn\_oldcore.py:1498: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
  if pd.api.types.is_categorical_dtype(vector):
c:\Users\tklas\anaconda3\envs\new_base\Lib\site-packages\seaborn\categorical.py:641: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  grouped_vals = vals.groupby(grouper)
c:\Users\tklas\anaconda3\envs\new_base\Lib\site-packages\seaborn\categorical.py:641: FutureWarning: 

c:\Users\tklas\anaconda3\envs\new_base\Lib\site-packages\seaborn\_oldcore.py:1498: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
  if pd.api.types.is_categorical_dtype(vector):
c:\Users\tklas\anaconda3\envs\new_base\Lib\site-packages\seaborn\_oldcore.py:1498: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
  if pd.api.types.is_categorical_dtype(vector):
c:\Users\tklas\anaconda3\envs\new_base\Lib\site-packages\seaborn\categorical.py:641: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  grouped_vals = vals.groupby(grouper)
c:\Users\tklas\anaconda3\envs\new_base\Lib\site-packages\seaborn\categorical.py:641: FutureWarning: 

#### Antwort:
Siehe avg_passenger_count_per_station_[...].png

#### Frage:
Wie viele aktive/inaktive Haltestellen/Haltepunkte gibt es in Haltestelle.csv und Haltepunkt.csv?

In [84]:
def create_pie_chart_active_inactive(save=False):
    labels = ['Active', 'Inactive']
    explode = (0, 0.05)
    colors = sns.color_palette('Set2')

    def custom_autopct(pct):
        return ('%1.1f%%' % pct) if pct > 0 else ''

    fig, axs = plt.subplots(1, 2, figsize=(12, 6))

    # Pie chart for Haltestellen
    sizes = [haltestelle_df['halt_ist_aktiv'].sum(), (~haltestelle_df['halt_ist_aktiv']).sum()]
    axs[0].pie(sizes, startangle=90, autopct=custom_autopct, explode=explode, colors=colors)
    axs[0].set_title('Haltestellen')
    axs[0].axis('equal')

    # Pie chart for Haltepunkte
    sizes = [haltepunkt_df['halt_punkt_ist_aktiv'].sum(), (~haltepunkt_df['halt_punkt_ist_aktiv']).sum()]
    axs[1].pie(sizes, startangle=90, autopct=custom_autopct, explode=explode, colors=colors)
    axs[1].set_title('Haltepunkte')
    axs[1].axis('equal')

    # Set main title and legend
    fig.suptitle(f'Percentage of Active and Inactive ({fahrzeiten_dir.split("/")[-2].split("_")[1]})', fontsize=16)
    fig.legend(labels, loc='upper center', bbox_to_anchor=(0.5, 0.9))
    
    if save:
        plt.savefig(f'../figures/exploration/{fahrzeiten_dir.split("/")[-2]}/percentage_active_inactive.png', bbox_inches='tight')
        plt.close()

In [85]:
create_pie_chart_active_inactive(save=True)

#### Antwort:
In Haltestellen.csv sind 100% der Haltestellen aktiv. In Haltepunkt.csv ist es verteilter. Siehe percentage_active_inactive.png

---

### Leere/Falsche Einträge
---
---

#### Frage:
Wie viele leere Einträge gibt es pro Spalte in einer CSV?

In [86]:
def create_bar_chart_empty_cells_percentage(df, save=False, df_name='placeholder'):
    empty_cells_percentages = {column: (sum_val := df[column].replace('', np.nan).isnull().sum(), sum_val / len(df)) for column in df.columns}

    # Check if there are any empty cells
    if all(num == 0 for num, _ in empty_cells_percentages.values()):
        print(f"There are no empty cells in any column of the {df_name} DataFrame.")
        return

    # Convert the dictionary to a DataFrame for easier plotting
    df_empty_cells = pd.DataFrame(empty_cells_percentages, index=['Empty Cells', 'Percentage']).T
    df_empty_cells = df_empty_cells.sort_values('Percentage', ascending=True)

    # Create the bar chart
    plt.figure(figsize=(10, 6))
    sns.barplot(x=df_empty_cells['Percentage'] * 100, y=df_empty_cells.index, palette='Set2', zorder=2)
    if df_name.startswith('Fahrzeiten_SOLL_IST') or df_name == 'haltepunkt' or df_name == 'haltestelle':
        plt.title(f'Empty Cells in Each Column of the {df_name} CSVs ({fahrzeiten_dir.split("/")[-2].split("_")[1]})')
    else:
        plt.title(f'Empty Cells in Each Column of the {df_name} CSV')
    plt.xlabel('Percentage (%)')
    plt.grid(axis='x', zorder=1)
    plt.xlim(0, 100)
    plt.xticks(np.arange(0, 101, 5))

    # Display the number of empty cells next to the bars
    for i, (num, perc) in enumerate(zip(df_empty_cells['Empty Cells'], df_empty_cells['Percentage'] * 100)):
        if num > 0:
            plt.text(perc, i, int(num), va='center')

    if save:
        if df_name.startswith('Fahrzeiten_SOLL_IST') or df_name == 'haltepunkt' or df_name == 'haltestelle':
            plt.savefig(f'../figures/exploration/{fahrzeiten_dir.split("/")[-2]}/empty_cells_percentage_{df_name}.png', bbox_inches='tight')
        else:
            plt.savefig(f'../figures/exploration/empty_cells_percentage_{df_name}.png', bbox_inches='tight')
        plt.close()

In [87]:
# create_bar_chart_empty_cells_percentage(passagierfrequenz_df, save=True, df_name='passagierfrequenz')
create_bar_chart_empty_cells_percentage(haltepunkt_df, save=True, df_name='haltepunkt')
create_bar_chart_empty_cells_percentage(haltestelle_df, save=True, df_name='haltestelle')

c:\Users\tklas\anaconda3\envs\new_base\Lib\site-packages\seaborn\_oldcore.py:1498: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
  if pd.api.types.is_categorical_dtype(vector):
c:\Users\tklas\anaconda3\envs\new_base\Lib\site-packages\seaborn\_oldcore.py:1498: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
  if pd.api.types.is_categorical_dtype(vector):
c:\Users\tklas\anaconda3\envs\new_base\Lib\site-packages\seaborn\_oldcore.py:1498: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
  if pd.api.types.is_categorical_dtype(vector):


There are no empty cells in any column of the haltestelle DataFrame.


c:\Users\tklas\anaconda3\envs\new_base\Lib\site-packages\seaborn\_oldcore.py:1498: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
  if pd.api.types.is_categorical_dtype(vector):
c:\Users\tklas\anaconda3\envs\new_base\Lib\site-packages\seaborn\_oldcore.py:1498: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
  if pd.api.types.is_categorical_dtype(vector):
c:\Users\tklas\anaconda3\envs\new_base\Lib\site-packages\seaborn\_oldcore.py:1498: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
  if pd.api.types.is_categorical_dtype(vector):


In [88]:
for df_name, df in fahrzeiten_dfs.items():
    create_bar_chart_empty_cells_percentage(df, save=True, df_name=df_name)

There are no empty cells in any column of the Fahrzeiten_SOLL_IST_20220102_20220108.csv DataFrame.
There are no empty cells in any column of the Fahrzeiten_SOLL_IST_20220109_20220115.csv DataFrame.
There are no empty cells in any column of the Fahrzeiten_SOLL_IST_20220116_20220122.csv DataFrame.
There are no empty cells in any column of the Fahrzeiten_SOLL_IST_20220123_20220129.csv DataFrame.
There are no empty cells in any column of the Fahrzeiten_SOLL_IST_20220130_20220205.csv DataFrame.
There are no empty cells in any column of the Fahrzeiten_SOLL_IST_20220206_20220212.csv DataFrame.
There are no empty cells in any column of the Fahrzeiten_SOLL_IST_20220213_20220219.csv DataFrame.
There are no empty cells in any column of the Fahrzeiten_SOLL_IST_20220220_20220226.csv DataFrame.
There are no empty cells in any column of the Fahrzeiten_SOLL_IST_20220227_20220305.csv DataFrame.
There are no empty cells in any column of the Fahrzeiten_SOLL_IST_20220306_20220312.csv DataFrame.
There are 

#### Antwort:
In haltestelle.csv und allen Fahrzeiten CSVs gibt es keine leeren Einträge. In haltepunkt.csv und passagierfrequenz.csv schon. Siehe empty_cells_percentage_[...].png

---

#### Frage:
Wie viele falsche Einträge gibt es pro Spalte in einer CSV?

In [89]:
def check_values(df, condition, message):
    if not condition(df).all():
        print(message)
        print('----------------------------------------------')
        return True
    return False

def count_values(df, condition):
    return (~condition(df)).sum()

grouped_conditions = {
    'Ungrouped': {
        'richtung contains values other than 1 or 2.': lambda df: df['richtung'].isin([1, 2]),
        'datum_von is greater than datum_nach.': lambda df: pd.to_datetime(df['datum_von'], format='%d.%m.%y') <= pd.to_datetime(df['datum_nach'], format='%d.%m.%y'),
        'seq_von is not 1 smaller than seq_nach.': lambda df: df['seq_von'] + 1 == df['seq_nach'],
    },
    'Between 0 and 86399': {
        'soll_an_von is not between 0 and 86399.': lambda df: (0 <= df['soll_an_von']) & (df['soll_an_von'] <= 86399),
        'soll_an_nach is not between 0 and 86399.': lambda df: (0 <= df['soll_an_nach']) & (df['soll_an_nach'] <= 86399),
    },
    'Less or equal': {
        'ist_an_von is greater than ist_ab_von.': lambda df: df['ist_an_von'] <= df['ist_ab_von'],
        'ist_an_nach1 is greater than ist_ab_nach.': lambda df: df['ist_an_nach1'] <= df['ist_ab_nach'],
        'soll_an_von is greater than soll_ab_von.': lambda df: df['soll_an_von'] <= df['soll_ab_von'],
        'soll_an_nach is greater than soll_ab_nach.': lambda df: df['soll_an_nach'] <= df['soll_ab_nach'],
    },
}

grouped_count_conditions = {
    'Greater than 86399': {
        'Entries in ist_an_von/ist_an_nach1 greater than 86399': lambda df: (df['ist_an_von'] <= 86399) & (df['ist_an_nach1'] <= 86399),
        'Entries in ist_ab_von/ist_ab_nach greater than 86399': lambda df: (df['ist_ab_von'] <= 86399) & (df['ist_ab_nach'] <= 86399),
        'Entries in soll_ab_von/soll_ab_nach greater than 86399': lambda df: (df['soll_ab_von'] <= 86399) & (df['soll_ab_nach'] <= 86399),
    },
    'Less than 0': {
        'Entries in ist_an_von/ist_an_nach1 less than 0': lambda df: (df['ist_an_von'] >= 0) & (df['ist_an_nach1'] >= 0),
        'Entries in ist_ab_von/ist_ab_nach less than 0': lambda df: (df['ist_ab_von'] >= 0) & (df['ist_ab_nach'] >= 0),
    },
}

def check_fahrzeiten_csvs():
    grouped_count_results = {group: {key: 0 for key in count_conditions.keys()} for group, count_conditions in grouped_count_conditions.items()}
    all_conditions_met = True

    for df in fahrzeiten_dfs.values():
        for group, conditions in grouped_conditions.items():
            for message, condition in conditions.items():
                if check_values(df, condition, message):
                    all_conditions_met = False
                    break
        for group, count_conditions in grouped_count_conditions.items():
            for message, condition in grouped_count_conditions[group].items():
                grouped_count_results[group][message] += count_values(df, condition)
            
    if all_conditions_met:
        print(f"All conditions are met")
        print('==============================================')

    for group, count_results in grouped_count_results.items():
        print(f'{group}:')
        print('----------------------------------------------')
        for message, count in count_results.items():
            print(f'{message}: {count}')
        print('==============================================')

In [90]:
# check_fahrzeiten_csvs()

Versuch zu schauen ob Tabellen matchen

In [91]:
# # halt_punkt_id_von and halt_punkt_id_nach from the fahrzeiten_dfs correspond to the haltpunkt_df['halt_punkt_id'] column
# # hald_id_von and halt_id_nach from the fahrzeiten_dfs correspond to the haltestelle_df['halt_id'] column
# # halt_id from the haltepunkt_df corresponds to the haltestelle_df['halt_id'] column
# # halt_diva_von and halt_diva_nach from the fahrzeiten_dfs correspond to the haltestelle_df['halt_diva'] column and it has to be the same combination with the halt_id_von and halt_id_nach to the halt_id and halt_diva columns in the haltestelle_df
# # halt_punkt_diva_von and halt_punkt_diva_nach from the fahrzeiten_dfs correspond to the haltepunkt_df['halt_punkt_diva'] column and it has to be the same combination with the halt_punkt_id_von and halt_punkt_id_nach to the halt_punkt_id and halt_punkt_diva columns in the haltepunkt_df
# # halt_kurz_von1 and halt_kurz_nach1 from the fahrzeiten_dfs correspond to the haltestelle_df['halt_kurz'] column and it has to be the same combination with the halt_id_von and halt_id_nach to the halt_id and halt_kurz columns in the haltestelle_df

# # Create a dictionary: {halt_punkt_id: (halt_punkt_diva, halt_id)}
# # And a dictionary: {halt_id: (halt_diva, halt_kurz)}
# halt_punkt_id_diva_id_dict = {}
# halt_id_diva_kurz_dict = {}
# for _, row in haltepunkt_df.iterrows():
#     halt_punkt_id_diva_id_dict[row['halt_punkt_id']] = (row['halt_punkt_diva'], row['halt_id'])
# for _, row in haltestelle_df.iterrows():
#     halt_id_diva_kurz_dict[row['halt_id']] = (row['halt_diva'], row['halt_kurz'])
    
# # Now check the conditions above
# all_conditions_met = True
# for df in fahrzeiten_dfs.values():
#     halt_punkt_id_von_diva_id = df[['halt_punkt_id_von', 'halt_punkt_diva_von', 'halt_id_von']].drop_duplicates()
#     halt_punkt_id_nach_diva_id = df[['halt_punkt_id_nach', 'halt_punkt_diva_nach', 'halt_id_nach']].drop_duplicates()
#     halt_id_von_diva_kurz = df[['halt_id_von', 'halt_diva_von', 'halt_kurz_von1']].drop_duplicates()
#     halt_id_nach_diva_kurz = df[['halt_id_nach', 'halt_diva_nach', 'halt_kurz_nach1']].drop_duplicates()
    
#     if not halt_punkt_id_von_diva_id.apply(lambda row: halt_punkt_id_diva_id_dict[row['halt_punkt_id_von']] == (row['halt_punkt_diva_von'], row['halt_id_von']), axis=1).all():
#         print('halt_punkt_id_von and halt_punkt_diva_von and halt_id_von are not consistent')
#         print('----------------------------------------------')
#         all_conditions_met = False
#     if not halt_punkt_id_nach_diva_id.apply(lambda row: halt_punkt_id_diva_id_dict[row['halt_punkt_id_nach']] == (row['halt_punkt_diva_nach'], row['halt_id_nach']), axis=1).all():
#         print('halt_punkt_id_nach and halt_punkt_diva_nach and halt_id_nach are not consistent')
#         print('----------------------------------------------')
#         all_conditions_met = False
#     if not halt_id_von_diva_kurz.apply(lambda row: halt_id_diva_kurz_dict[row['halt_id_von']] == (row['halt_diva_von'], row['halt_kurz_von1']), axis=1).all():
#         print('halt_id_von and halt_diva_von and halt_kurz_von1 are not consistent')
#         print('----------------------------------------------')
#         all_conditions_met = False
#     if not halt_id_nach_diva_kurz.apply(lambda row: halt_id_diva_kurz_dict[row['halt_id_nach']] == (row['halt_diva_nach'], row['halt_kurz_nach1']), axis=1).all():
#         print('halt_id_nach and halt_diva_nach and halt_kurz_nach1 are not consistent')
#         print('----------------------------------------------')
#         all_conditions_met = False
        
# if all_conditions_met:
#     print(f"All conditions are met")
    # print('==============================================')

#### Antwort:
Die Einträge in den Fahrzeiten CSVs sind alle korrekt. Es gibt Einträge, die den Hinweisen entsprechen.

---

### Wie oft ist ... in den Fahrzeiten CSVs?
---
---

#### Frage:
Wie oft ist eine Bahn (linie) in den Fahrzeiten CSVs?

In [92]:
def create_bar_chart_linien_counts(save=False, values=10):
    linien_counts = {}

    for df in fahrzeiten_dfs.values():
        size = df.groupby('linie').size()
        for linie, count in size.items():
            if linie in linien_counts:
                linien_counts[linie] += count
            else:
                linien_counts[linie] = count
                
    df_linien_counts = pd.DataFrame(linien_counts.items(), columns=['Line', 'Count'])
    
    if values > df_linien_counts.shape[0]:
        values = df_linien_counts.shape[0]
    
    df_linien_counts = df_linien_counts.sort_values('Count', ascending=False).head(values)

    # Create the bar chart
    fig, ax = plt.subplots(figsize=(10, 6))
    sns.barplot(x=df_linien_counts['Line'], y=df_linien_counts['Count'], order=df_linien_counts['Line'], ax=ax, palette='Set2', zorder=2)
    ax.yaxis.set_major_formatter(ticker.FuncFormatter(lambda x, pos: '{:,.0f}'.format(x)))
    plt.title(f'Top {values} Most Frequent Lines in Fahrzeiten CSVs ({fahrzeiten_dir.split("/")[-2].split("_")[1]})')
    plt.xlabel('Line')
    plt.ylabel('Amount of Entries')
    plt.grid(axis='y', zorder=1)
    
    if save:
        plt.savefig(f'../figures/exploration/{fahrzeiten_dir.split("/")[-2]}/linien_count.png', bbox_inches='tight')
        plt.close()

In [93]:
create_bar_chart_linien_counts(save=True, values=20)

c:\Users\tklas\anaconda3\envs\new_base\Lib\site-packages\seaborn\_oldcore.py:1498: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
  if pd.api.types.is_categorical_dtype(vector):
c:\Users\tklas\anaconda3\envs\new_base\Lib\site-packages\seaborn\_oldcore.py:1498: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
  if pd.api.types.is_categorical_dtype(vector):


#### Antwort:
Siehe linien_count.png

---

#### Frage:
Wie oft ist eine Haltestelle in den Fahrzeiten CSVs?

In [94]:
def create_bar_chart_haltestellen_counts(save=False, values=10):
    haltestellen_counts = {}

    for df in fahrzeiten_dfs.values():
        size = df.groupby('halt_id_von').size()
        for halt, count in size.items():
            haltestellen_name = haltestelle_df[haltestelle_df['halt_id'] == halt]['halt_lang'].values[0]
            if halt in haltestellen_counts:
                haltestellen_counts[haltestellen_name] += count
            else:
                haltestellen_counts[haltestellen_name] = count
                
    df_haltestellen_counts = pd.DataFrame(haltestellen_counts.items(), columns=['Haltestelle', 'Count'])
    df_haltestellen_counts = df_haltestellen_counts.sort_values('Count', ascending=False).head(values)

    # Create the horizontal bar chart
    fig, ax = plt.subplots(figsize=(10, 6))
    sns.barplot(x=df_haltestellen_counts['Count'], y=df_haltestellen_counts['Haltestelle'], order=df_haltestellen_counts['Haltestelle'], ax=ax, palette='Set2', zorder=2)
    ax.xaxis.set_major_formatter(ticker.FuncFormatter(lambda x, pos: '{:,.0f}'.format(x)))
    plt.title(f'Top {values} Most Frequent Bus/Train Stops in Fahrzeiten CSVs ({fahrzeiten_dir.split("/")[-2].split("_")[1]})')
    plt.xlabel('Amount of Entries')
    plt.ylabel('')
    plt.grid(axis='x', zorder=1)
    
    if save:
        plt.savefig(f'../figures/exploration/{fahrzeiten_dir.split("/")[-2]}/haltestellen_count.png', bbox_inches='tight')
        plt.close()

In [95]:
create_bar_chart_haltestellen_counts(save=True, values=20)

c:\Users\tklas\anaconda3\envs\new_base\Lib\site-packages\seaborn\_oldcore.py:1498: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
  if pd.api.types.is_categorical_dtype(vector):
c:\Users\tklas\anaconda3\envs\new_base\Lib\site-packages\seaborn\_oldcore.py:1498: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
  if pd.api.types.is_categorical_dtype(vector):


#### Antwort:
Siehe haltestellen_count.png

---

#### Frage:
Wie oft ist eine Stadt in den Fahrzeiten CSVs?

In [96]:
def create_bar_chart_staedte_count(save=False, values=10):
    staedte_counts = {}
    
    for df in fahrzeiten_dfs.values():
        halt_diva = df['halt_id_von'].unique()
        haltestellen_name = haltestelle_df[haltestelle_df['halt_id'].isin(halt_diva)]['halt_lang'].unique()
        stadt_name = map(lambda x: x.split(',')[0], haltestellen_name)
        for stadt in stadt_name:
            if stadt in staedte_counts:
                staedte_counts[stadt] += 1
            else:
                staedte_counts[stadt] = 1
                
    df_staedte_counts = pd.DataFrame(staedte_counts.items(), columns=['Stadt', 'Count'])
    df_staedte_counts = df_staedte_counts.sort_values('Count', ascending=False).head(values)
    
    # Create the horizontal bar chart
    fig, ax = plt.subplots(figsize=(10, 6))
    sns.barplot(x=df_staedte_counts['Count'], y=df_staedte_counts['Stadt'], order=df_staedte_counts['Stadt'], ax=ax, palette='Set2', zorder=2)
    ax.xaxis.set_major_formatter(ticker.FuncFormatter(lambda x, pos: '{:,.0f}'.format(x)))
    plt.title(f'Top {values} Most Frequent Cities in Fahrzeiten CSVs ({fahrzeiten_dir.split("/")[-2].split("_")[1]})')
    plt.xlabel('Amount of Entries')
    plt.ylabel('')
    plt.xticks(np.arange(0, 24001, 2000))
    plt.grid(axis='x', zorder=1)

    # Add the number of the value next to the bar
    for i, v in enumerate(df_staedte_counts['Count']):
        ax.text(v + 0.5, i, str(v), color='black', va='center')

    if save:
        plt.savefig(f'../figures/exploration/{fahrzeiten_dir.split("/")[-2]}/staedte_count.png', bbox_inches='tight')
        plt.close()

In [97]:
create_bar_chart_staedte_count(save=True, values=20)

c:\Users\tklas\anaconda3\envs\new_base\Lib\site-packages\seaborn\_oldcore.py:1498: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
  if pd.api.types.is_categorical_dtype(vector):
c:\Users\tklas\anaconda3\envs\new_base\Lib\site-packages\seaborn\_oldcore.py:1498: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
  if pd.api.types.is_categorical_dtype(vector):


#### Antwort:
Siehe staedte_count.png

---

#### Frage:
Wie viele Einträge gibt es pro Betriebsdatum in den Fahrzeiten CSVs?

In [98]:
betriebsdatum_counts = {}

for df in fahrzeiten_dfs.values():
    size = df.groupby('betriebsdatum').size()
    for betriebsdatum, count in size.items():
        if betriebsdatum in betriebsdatum_counts:
            betriebsdatum_counts[betriebsdatum] += count
        else:
            betriebsdatum_counts[betriebsdatum] = count

df_betriebsdatum_counts = pd.DataFrame(betriebsdatum_counts.items(), columns=['Betriebsdatum', 'Count'])
df_betriebsdatum_counts = df_betriebsdatum_counts.sort_values('Count', ascending=False)
df_betriebsdatum_counts['Betriebsdatum'] = pd.to_datetime(df_betriebsdatum_counts['Betriebsdatum'], format='%d.%m.%y')

def show_betriebsdatum_counts():
    # Erstellen einer Dash-App
    app = dash.Dash(__name__)

    # Layout des Dash-Dashboards
    app.layout = html.Div([
        html.Div([
            html.Button(f"Overview\n{df_betriebsdatum_counts['Count'].sum():,.0f}",
                        id='button-overview', n_clicks=1, style={'display': 'inline-block', 'white-space': 'pre-line', 'width': '10%'}),
            html.Div([
                html.Button(f"{calendar.month_name[i]}\n{df_betriebsdatum_counts[df_betriebsdatum_counts['Betriebsdatum'].dt.month == i]['Count'].sum():,.0f}",
                            id=f'button-{i}', n_clicks=0, style={'display': 'inline-block', 'white-space': 'pre-line', 'width': '8.33%'})
                for i in range(1, 13)
            ], style={'text-align': 'center'}),
        ], style={'text-align': 'center'}),
        dcc.Graph(id='calendar-plot'),
    ], className='container')

    @app.callback(
        Output('calendar-plot', 'figure'),
        [Input('button-overview', 'n_clicks')] + [Input(f'button-{i}', 'n_clicks') for i in range(1, 13)]
    )
    def update_calendar(n_overview, *n_clicks):
        ctx = dash.callback_context
        if not ctx.triggered:
            month_num = 0
        else:
            trigger_id = ctx.triggered[0]['prop_id'].split('.')[0]
            if trigger_id == 'button-overview':
                month_num = 0
            else:
                month_num = int(trigger_id.split('-')[-1])

        if month_num == 0:
            df_grouped = df_betriebsdatum_counts.groupby(df_betriebsdatum_counts['Betriebsdatum'].dt.month)['Count'].sum()
            fig = {
                'data': [
                    {'x': [calendar.month_name[i] for i in df_grouped.index], 'y': df_grouped, 'type': 'bar', 'name': 'Anzahl Zeilen'},
                ],
                'layout': {
                    'title': 'Amount of Entries for every Month in Fahrzeiten CSVs',
                    'xaxis': {'title': 'Month'},
                    'yaxis': {'title': 'Entries'}
                }
            }
        else:
            filtered_df = df_betriebsdatum_counts[df_betriebsdatum_counts['Betriebsdatum'].dt.month == month_num]
            fig = {
                'data': [
                    {'x': filtered_df['Betriebsdatum'], 'y': filtered_df['Count'], 'type': 'bar', 'name': 'Anzahl Zeilen'},
                ],
                'layout': {
                    'title': f'Amount of Entries in {calendar.month_name[month_num]} in Fahrzeiten CSVs',
                    'xaxis': {'title': 'Day'},
                    'yaxis': {'title': 'Entries'}
                }
            }

        return fig

    if __name__ == '__main__':
        app.run_server(debug=True)

In [99]:
# show_betriebsdatum_counts()

#### Antwort:
Siehe Interaktion

---

#### Frage:
Wie viele Fahrzeuge gibt es in den Fahrzeiten CSVs?

In [100]:
# Wie viele Fahrzeuge sind in den Fahrzeiten CSVs enthalten?
fahrzeuge = set()

for df in fahrzeiten_dfs.values():
    fahrzeuge.update(df['fahrzeug'].unique())

print(f'Anzahl Fahrzeuge: {len(fahrzeuge)}')
print(fahrzeuge)

Anzahl Fahrzeuge: 576
{2048, 2049, 2050, 2051, 2052, 2053, 2054, 2055, 2056, 2057, 2059, 2061, 2062, 2063, 2064, 2065, 2066, 2067, 2068, 2069, 2070, 2071, 2072, 10264, 2074, 2075, 2076, 2077, 2078, 2079, 2080, 2081, 2082, 2083, 2084, 2085, 2086, 10276, 2088, 2089, 2090, 2091, 2092, 2093, 2094, 2095, 2096, 2097, 2098, 2099, 2100, 2101, 2102, 2103, 2104, 2105, 2106, 2107, 2108, 2109, 2110, 2111, 2112, 2113, 2114, 2115, 2116, 2117, 2118, 2119, 2120, 2121, 10312, 10310, 10313, 10331, 10332, 10333, 10334, 10335, 10336, 10337, 10338, 10340, 10341, 10342, 10343, 10344, 10345, 10346, 10347, 10203, 10208, 10401, 10402, 10403, 10404, 10405, 10406, 10407, 10408, 10409, 10410, 10411, 10412, 10413, 10414, 10415, 10416, 10417, 10418, 10419, 10420, 10421, 10422, 10423, 10424, 10425, 10426, 10427, 10428, 10429, 10430, 10431, 2073, 10440, 10441, 10442, 10443, 10444, 10445, 10446, 10447, 10448, 10450, 10451, 10452, 10453, 10454, 10455, 10456, 10457, 10458, 10459, 10460, 10461, 10462, 10463, 10464, 10465

#### Antwort:
Es gibt 576 Fahrzeuge.

---

### Weiteres
---
---

#### Frage:
Welcher Kanton hat die meisten Einträge in der Passagierfrequenz CSV?

In [101]:
canton_counts = {}

cantons = passagierfrequenz_df['kt_ct_cantone'].unique()
for canton in cantons:
    canton_counts[canton] = passagierfrequenz_df[passagierfrequenz_df['kt_ct_cantone'] == canton].shape[0]

canton_dict = {
    'AG': ('Aargau', [47.387666, 8.247164]),
    'BE': ('Bern', [46.948020, 7.447433]),
    'FR': ('Fribourg', [46.806467, 7.161594]),
    'GE': ('Genève', [46.204391, 6.143158]),
    'GL': ('Glarus', [47.040182, 9.067609]),
    'GR': ('Graubünden', [46.656987, 9.578027]),
    'JU': ('Jura', [47.350000, 7.350000]),
    'LU': ('Luzern', [47.050000, 8.300000]),
    'NE': ('Neuchâtel', [46.991789, 6.930000]),
    'SG': ('St. Gallen', [47.423058, 9.377083]),
    'SH': ('Schaffhausen', [47.697320, 8.634910]),
    'SZ': ('Schwyz', [47.020000, 8.650000]),
    'SO': ('Solothurn', [47.206667, 7.537778]),
    'TG': ('Thurgau', [47.566667, 9.166667]),
    'TI': ('Ticino', [46.331734, 8.800452]),
    'UR': ('Uri', [46.900000, 8.633333]),
    'VS': ('Valais', [46.191079, 7.737933]),
    'VD': ('Vaud', [46.533333, 6.666667]),
    'ZG': ('Zug', [47.166667, 8.516667]),
    'ZH': ('Zürich', [47.366667, 8.550000]),
    'AR': ('Appenzell Ausserrhoden', [47.366667, 9.300000]),
    'AI': ('Appenzell Innerrhoden', [47.316667, 9.416667]),
    'BS': ('Basel-Stadt', [47.566667, 7.600000]),
    'BL': ('Basel-Landschaft', [47.450000, 7.750000]),
    'OW': ('Obwalden', [46.900000, 8.250000]),
    'NW': ('Nidwalden', [46.950000, 8.383333])
}

canton_counts = {canton_dict[k][0]: v for k, v in canton_counts.items() if k in canton_dict.keys() and v > 0}

canton_counts_df = pd.DataFrame.from_dict(canton_counts, orient='index', columns=['count'])

In [102]:
def create_bar_chart_canton_counts(save=False):
    # Filter rows where jahr_annee_anno is 2022
    passagierfrequenz_df_2022 = passagierfrequenz_df.loc[passagierfrequenz_df['jahr_annee_anno'] == 2022]

    canton_counts = passagierfrequenz_df_2022['kt_ct_cantone'].value_counts()
    canton_counts = canton_counts[canton_counts.index.isin(canton_dict.keys())]

    # Replace abbreviations with full names
    canton_counts.index = [canton_dict[canton][0] for canton in canton_counts.index]

    # Create the bar chart
    fig, ax = plt.subplots(figsize=(10, 6))
    sns.barplot(y=canton_counts.index, x=canton_counts, order=canton_counts.index, ax=ax, palette='Set2', zorder=2)
    ax.xaxis.set_major_formatter(ticker.FuncFormatter(lambda x, pos: '{:,.0f}'.format(x)))
    plt.title(f'Amount of Stations for every Canton in Passagierfrequenz CSV')
    plt.ylabel('')
    plt.xlabel('Amount of Entries')
    plt.grid(axis='x', zorder=1)

    if save:
        plt.savefig('../figures/exploration/canton_count.png', bbox_inches='tight')
        plt.close()

In [103]:
create_bar_chart_canton_counts(save=True)

c:\Users\tklas\anaconda3\envs\new_base\Lib\site-packages\seaborn\_oldcore.py:1498: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
  if pd.api.types.is_categorical_dtype(vector):
c:\Users\tklas\anaconda3\envs\new_base\Lib\site-packages\seaborn\_oldcore.py:1498: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
  if pd.api.types.is_categorical_dtype(vector):


In [104]:
def create_canton_map():
    canton_map = folium.Map(location=[46.8, 8.33], zoom_start=8, tiles='cartodbpositron')
    folium.Choropleth(
        geo_data='../raw_data/switzerland.geojson',
        data=canton_counts_df,
        columns=[canton_counts_df.index, 'count'],
        key_on='feature.properties.NAME',
        fill_color='YlGn',
        fill_opacity=0.7,
        line_opacity=0.2,
        legend_name='Amount of entries in passagierfrequenz.csv',
        legend_kwds={'loc': 'bottom left'}
    ).add_to(canton_map)

    # Create a reverse dictionary that maps the full canton names to their abbreviations
    reverse_canton_dict = {v[0]: k for k, v in canton_dict.items()}

    # # For each canton, add a marker to the map
    # for canton, count in canton_counts.items():
    #     # Get the abbreviation of the canton
    #     canton_abbr = reverse_canton_dict[canton]

    #     # Get the coordinates of the canton
    #     coords = canton_dict[canton_abbr][1]

    #     # Create a Marker and add it to the map
    #     folium.Marker(
    #         location=coords,
    #         popup=f'{canton}\n{count}',
    #         icon=folium.Icon(icon='info-sign')
    #     ).add_to(canton_map)

    return canton_map

In [105]:
# create_canton_map()

#### Antwort:
Siehe Interaktion

---

#### Frage:
An welchen Wochentagen gibt es die meisten Einträge in den Fahrzeiten CSVs?

In [106]:
def create_bar_chart_weekday_counts(save=False):
    weekday_counts = {}

    for betriebsdatum, count in betriebsdatum_counts.items():
        weekday = calendar.day_name[datetime.strptime(betriebsdatum, '%d.%m.%y').weekday()]
        if weekday in weekday_counts:
            weekday_counts[weekday] += count
        else:
            weekday_counts[weekday] = count
            
    df_weekday_counts = pd.DataFrame(weekday_counts.items(), columns=['Weekday', 'Count'])
    weekday_to_num = {'Monday': 1, 'Tuesday': 2, 'Wednesday': 3, 'Thursday': 4, 'Friday': 5, 'Saturday': 6, 'Sunday': 7}
    df_weekday_counts['Weekday_num'] = df_weekday_counts['Weekday'].map(weekday_to_num)
    df_weekday_counts = df_weekday_counts.sort_values('Weekday_num')

    # Create the bar chart
    fig, ax = plt.subplots(figsize=(10, 6))
    sns.barplot(x=df_weekday_counts['Weekday'], y=df_weekday_counts['Count'], order=df_weekday_counts['Weekday'], ax=ax, palette='Set2', zorder=2)
    ax.yaxis.set_major_formatter(ticker.FuncFormatter(lambda x, pos: '{:,.0f}'.format(x)))
    plt.title(f'Amount of Entries for every Weekday in Fahrzeiten CSVs ({fahrzeiten_dir.split("/")[-2].split("_")[1]})')
    plt.xlabel('')
    plt.ylabel('Entries')
    plt.grid(axis='y', zorder=1)
    
    if save:
        plt.savefig(f'../figures/exploration/{fahrzeiten_dir.split("/")[-2]}/weekday_count.png', bbox_inches='tight')
        plt.close()

In [107]:
create_bar_chart_weekday_counts(save=True)

c:\Users\tklas\anaconda3\envs\new_base\Lib\site-packages\seaborn\_oldcore.py:1498: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
  if pd.api.types.is_categorical_dtype(vector):
c:\Users\tklas\anaconda3\envs\new_base\Lib\site-packages\seaborn\_oldcore.py:1498: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
  if pd.api.types.is_categorical_dtype(vector):


#### Antwort:
Siehe weekday_count.png

---

#### Frage:
Wie viele unterschiedliche Einträge gibt es pro Spalte in den Fahrzeiten CSVs?

In [108]:
def create_bar_chart_unique_entries_per_column(df, save=True, df_name='placeholder'):
    unique_entries_per_column = {}

    for column in df.columns:
        unique_entries_per_column[column] = df[column].unique().shape[0]

    df_unique_entries_per_column = pd.DataFrame(unique_entries_per_column.items(), columns=['Column', 'Count'])
    
    # Add a new column that indicates whether each column only contains unique values
    total_rows = df.shape[0]
    df_unique_entries_per_column['Is Unique'] = df_unique_entries_per_column['Count'] == total_rows
    
    # Sort the dataframe in ascending order
    df_unique_entries_per_column = df_unique_entries_per_column.sort_values('Count')

    # Create the bar chart
    fig, ax = plt.subplots(figsize=(10, 4))
    sns.barplot(y=df_unique_entries_per_column['Column'], x=df_unique_entries_per_column['Count'], order=df_unique_entries_per_column['Column'], ax=ax, palette='Set2', zorder=2)
    ax.xaxis.set_major_formatter(ticker.FuncFormatter(lambda x, pos: '{:,.0f}'.format(x)))
    if df_name == 'haltepunkt' or df_name == 'haltestelle':
        plt.title(f'Unique Entries per Column in {df_name.title()} ({fahrzeiten_dir.split("/")[-2].split("_")[1]})')
    else:
        plt.title(f'Unique Entries per Column in {df_name.title()} (Total Rows: {total_rows})')
    plt.ylabel('Column')
    plt.xlabel('Unique Entries')
    plt.grid(axis='x', zorder=1)
    
    # Add a vertical line at the position of total_rows
    plt.axvline(x=total_rows, color='red', linestyle='--', linewidth=1)
        
    if save:
        if df_name == 'haltepunkt' or df_name == 'haltestelle':
            plt.savefig(f'../figures/exploration/{fahrzeiten_dir.split("/")[-2]}/unique_entries_per_column_{df_name}.png', bbox_inches='tight')
        else:
            plt.savefig(f'../figures/exploration/unique_entries_per_column_{df_name}.png', bbox_inches='tight')
        plt.close()

In [109]:
# create_bar_chart_unique_entries_per_column(passagierfrequenz_df, save=True, df_name='passagierfrequenz')
create_bar_chart_unique_entries_per_column(haltepunkt_df, save=True, df_name='haltepunkt')
create_bar_chart_unique_entries_per_column(haltestelle_df, save=True, df_name='haltestelle')

c:\Users\tklas\anaconda3\envs\new_base\Lib\site-packages\seaborn\_oldcore.py:1498: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
  if pd.api.types.is_categorical_dtype(vector):
c:\Users\tklas\anaconda3\envs\new_base\Lib\site-packages\seaborn\_oldcore.py:1498: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
  if pd.api.types.is_categorical_dtype(vector):
c:\Users\tklas\anaconda3\envs\new_base\Lib\site-packages\seaborn\_oldcore.py:1498: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
  if pd.api.types.is_categorical_dtype(vector):
c:\Users\tklas\anaconda3\envs\new_base\Lib\site-packages\seaborn\_oldcore.py:1498: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(d

#### Antwort:
Siehe unique_entries_per_column_[...].png

---